In [3]:
import pickle
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm 
import pandas as pd

In [2]:
import datasets
news_train_dataset = datasets.load_dataset('nlpHakdang/aihub-news30k',  data_files = "train_news_text.csv", use_auth_token='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')
news_train_dataset


Using custom data configuration aihub-news30k-7dc905421859817c


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /opt/ml/.cache/huggingface/datasets/csv/aihub-news30k-7dc905421859817c/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', '0'],
        num_rows: 27303
    })
})

In [13]:
news_train_df = pd.DataFrame( news_train_dataset['train'] )
news_train_df.head()

,Unnamed: 0,0
0,0,여수광양항만공사(사장 박희석)는 14일부터 20일까지 광양항 배후 투자 유치를 위해...
1,1,포스코 광양제철소(소장 임학동)가 자동차 강판 생산공장인 제3냉연공장을 최신 설비로...
2,2,"여수광양항만공사(사장 밤 희석, 이하 공사)는 2018년도 승진 전 보 인사를 단행..."
3,3,광양시는 경쟁력을 갖춘 풍요로운 농어촌 건설을 위해 원예작물 신기술 보급사업을 통한...
4,4,포스코 광양제철소(소장 임학동)가 ‘사랑 나눔 헌혈 행사’를 실시하며 생명 나눔을 ...


In [14]:
news_train_df = news_train_df.drop('Unnamed: 0', axis = 1)
news_train_df.head()

,0
0,여수광양항만공사(사장 박희석)는 14일부터 20일까지 광양항 배후 투자 유치를 위해...
1,포스코 광양제철소(소장 임학동)가 자동차 강판 생산공장인 제3냉연공장을 최신 설비로...
2,"여수광양항만공사(사장 밤 희석, 이하 공사)는 2018년도 승진 전 보 인사를 단행..."
3,광양시는 경쟁력을 갖춘 풍요로운 농어촌 건설을 위해 원예작물 신기술 보급사업을 통한...
4,포스코 광양제철소(소장 임학동)가 ‘사랑 나눔 헌혈 행사’를 실시하며 생명 나눔을 ...


In [37]:
train = news_train_df

In [8]:
# with open("news_dict_nouns.pickle","wb") as f:
#     pickle.dump(nouns, f)
with open("news_dict_nouns.pickle","rb") as f :
    nouns = pickle.load(f)

In [9]:
# len_list = []
# for key, val in nouns.items():
#     len_list.append((key, len(val)))

In [10]:
# del(nouns['전방'])
# del(pre_dict['전방'])

In [11]:
# del(pre_dict['대상'])

In [12]:
data = sorted(len_list, key = lambda x: x[1],reverse=True)

In [13]:
pre_dict = {}
for key, vals in nouns.items():
    pre_list = []
    if len(vals) == 0:
        continue
    for val in vals:
        pre_list.append(val.replace(key,"[MASK]"))
    pre_dict[key] = pre_list

In [14]:
import pandas as pd

In [15]:
col_list = ["corp","plain","masked"]

In [16]:
data_list = []
for key in nouns.keys():
    for idx in range(len(nouns[key])):
        data_list.append([key, nouns[key][idx],pre_dict[key][idx]])

In [17]:
len(data_list)

145240

In [18]:
df = pd.DataFrame(data_list, columns=col_list)

In [19]:
# with open("dataset.pickle","wb") as f:
#     pickle.dump(df, f)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
train, test, _, _= train_test_split(
    df,df['corp'], test_size=0.3, random_state=42)

In [22]:
valid, test, _, _= train_test_split(
    test,test['corp'], test_size=0.5, random_state=42)

In [23]:
label_to_corp = {idx:list(nouns.keys())[idx] for idx in range(len(list(nouns.keys())))}
corp_to_label = {val:idx for idx, val in label_to_corp.items()}

In [24]:
news_length = [len(txt) for txt in df['plain'].values]
max(news_length)

13678

In [23]:
import datasets
dartdataset = datasets.load_dataset('nlpHakdang/beneficiary',  data_files = "dart_v3_01.csv", use_auth_token='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')


Using custom data configuration beneficiary-15e09db052e9f529
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/beneficiary-15e09db052e9f529/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
import re
def preprocessing_only_kr(s): 
        hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return ' '.join(result.split())

def preprocessing(s): 
        hangul = re.compile('[^ a-zA-Z0-9ㄱ-ㅣ가-힣+]')
        result = hangul.sub('', s)
        return ' '.join(result.split())

In [31]:
preprocess = [preprocessing(txt) for txt in dartdataset['train']['사업의 개요']]
dartdata = [dartdataset['train']['기업 이름'],preprocess]
dartdf = pd.DataFrame (dartdata).transpose()
dartdf.columns = ['corp', 'txt']
len(dartdf)

2410

### Stride Code
#### DART
- 데이터의 형태 colum = ['corp', 'txt']
    - corp : 기업이름, txt : 사업의 개요
- corp_to_label : 회사이름을 정수로 바꾸는 딕셔너리

In [55]:
def dart_split(df : pd.DataFrame) -> pd.DataFrame : 
    data_list2 = []
    for idx in tqdm(range(len(df))):
        stride = []
        txt = df.iloc[idx]['txt']
        if len(txt) < 512:
            # if df.iloc[idx]['corp'] not in corp_to_label:
                # continue
            data_list2.append([df.iloc[idx]['corp'],df.iloc[idx]['txt']])
        else:
            for pointer in range(len(txt)//300):
                if pointer == len(txt)//300-1:
                    split_txt = txt[-400:]
                elif pointer != 0:
                    split_txt = txt[(pointer*300)-100:((pointer+1)*300)+100]
                elif pointer == 0:
                    split_txt = txt[:400]
                # elif df.iloc[idx]['corp'] not in corp_to_label:
                #     continue
                data_list2.append([df.iloc[idx]['corp'],split_txt])
    return pd.DataFrame(data_list2,columns=['corp', 'txt'])

#### News
- 데이터의 형태 colum = ['corp', 'plain', 'maked']
    - corp : 기업이름, plain : 원본텍스트, masked:마스크된 텍스트
- corp_to_label : 회사이름을 정수로 바꾸는 딕셔너리

In [45]:
def bert_split(df : pd.DataFrame) -> pd.DataFrame:
    data_list2 = []
    for idx in tqdm(range(len(df))):
        stride = []
        txt = df.iloc[idx]['0']
        if txt == None:
            continue
        if len(txt) < 512:
            data_list2.append([df.iloc[idx]['0']])
        else:
            for pointer in range(len(txt)//300):
                if pointer == len(txt)//300-1:
                    split_txt = txt[-400:]
                if pointer != 0:
                    split_txt = txt[(pointer*300)-100:((pointer+1)*300)+100]
                if pointer == 0:
                    split_txt = txt[:400]
                data_list2.append([split_txt])
    return pd.DataFrame(data_list2,columns=['divided'])

In [46]:
train_df = bert_split(train)
# valid_df = bert_split(valid)
# test_df = bert_split(test)


100%|██████████| 27303/27303 [00:02<00:00, 11813.85it/s]


In [47]:
train_df.head()

,divided
0,여수광양항만공사(사장 박희석)는 14일부터 20일까지 광양항 배후 투자 유치를 위해...
1,은 먼저 광양항 서측 배후 푸드존 투자유치를 위해 중국 윈난성 소재 커피 원재료 공...
2,포스코 광양제철소(소장 임학동)가 자동차 강판 생산공장인 제3냉연공장을 최신 설비로...
3,판 중 높은 품질기술력을 요구하는 자동차 외판 생산 증대에 초점을 두고 진행됐다. ...
4,"여수광양항만공사(사장 밤 희석, 이하 공사)는 2018년도 승진 전 보 인사를 단행..."


In [ ]:
train.head()

,corp,plain,masked
131090,한화,금융권이 주 52시간 근무제의 법적 시행을 일주일 앞두고 마지막 점검 중이다은행권은...,금융권이 주 52시간 근무제의 법적 시행을 일주일 앞두고 마지막 점검 중이다은행권은...
111221,태양,한국화학연구원이하 화학연이 차세대 태양전지로 주목받는 페로브스카이트 태양전지 최고 ...,한국화학연구원이하 화학연이 차세대 [MASK]전지로 주목받는 페로브스카이트 [MAS...
40789,현대건설,서울 용산 한남3구역 재개발 사업이 국토교통부 서울시의 모호한 방침 때문에 혼란에 ...,서울 용산 한남3구역 재개발 사업이 국토교통부 서울시의 모호한 방침 때문에 혼란에 ...
97947,대상,에이스침대가 봄을 맞아 다양한 브랜드 제품을 한 자리에서 만날 수 있는 에이스 브랜...,에이스침대가 봄을 맞아 다양한 브랜드 제품을 한 자리에서 만날 수 있는 에이스 브랜...
97517,대상,이데일리 김지섭 기자 JW그룹 신입사원들이 새해를 맞아 뜻 깊은 봉사활동을 진행했다...,이데일리 김지섭 기자 JW그룹 신입사원들이 새해를 맞아 뜻 깊은 봉사활동을 진행했다...


In [56]:
dartdf.head()
split_dart_df = dart_split(dartdf)

100%|██████████| 2410/2410 [00:01<00:00, 1765.16it/s]


In [57]:
split_dart_df.head()

,corp,txt
0,엑세스바이오,당사는 체외진단 기술을 토대로 면역화학진단 바이오센서 분자진단 기술을 기반으로 진단...
1,엑세스바이오,어 의약품 제조기준에 규제를 받고 있습니다 체외진단 시장은 인구 고령화 및 감염성 ...
2,엑세스바이오,019년 신개발 의료기기 전망분석 보고서 에 따르면 체외진단기기 세계시장은 2015...
3,엑세스바이오,FDA로부터 긴급사용승인 Emergency Use Authorization 을 획...
4,지오엠씨,가 영업의 현황 당사는 30년간 지속적으로 진행해 오고 있는 엠씨스퀘어 사업 부문과...


In [34]:
# len(train), len(test), len(train_df), len(test_df), len(dartdf), len(split_dart_df)

In [35]:
# test_df.iloc[0:3]['masked'][0]

In [36]:
# test_df.iloc[0:3]['masked'][1]

In [37]:
# test_df.iloc[0:3]['masked'][2]

In [38]:
# test_df

In [58]:
split_dart = split_dart_df.rename_axis('id').reset_index()

In [40]:
# id_to_corp = {}
# for idx in range(len(split_dart)):
#     id_to_corp[split_dart.iloc[idx]['id']] = split_dart.iloc[idx]['corp']

In [59]:
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel
)
import transformers

In [60]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [61]:
# train_df

In [63]:
dart_tok = tokenizer( list(split_dart_df['txt']) ,padding = "max_length", return_tensors = "pt", truncation = True) 
train_tok = tokenizer( list(train_df['divided']) ,padding = "max_length", return_tensors = "pt", truncation = True)
# valid_tok = tokenizer( list(valid_df['masked']) ,padding = "max_length", return_tensors = "pt", truncation = True)
# test_tok = tokenizer(list(test_df['masked']),padding = "max_length", return_tensors = "pt", truncation = True)

In [ ]:
# train_corp_code = torch.tensor(train_df['corp'])
# valid_corp_code = torch.tensor(valid_df['corp'])
# test_corp_code = torch.tensor(test_df['corp'])

In [ ]:
# dart_tok['input_ids'].shape#, train_tok['input_ids'].shape, train_corp_code.shape

In [ ]:
# dart_tok.keys()

In [64]:
from torch.utils.data import DataLoader, TensorDataset

In [65]:
news_set = TensorDataset(
    train_tok['input_ids'], train_tok['token_type_ids'], train_tok['attention_mask'])
    # train_corp_code)
dart_set = TensorDataset(
    dart_tok['input_ids'], dart_tok['token_type_ids'], dart_tok['attention_mask'])
# valid_set = TensorDataset(
#     valid_tok['input_ids'], valid_tok['token_type_ids'], valid_tok['attention_mask'], \
#     valid_corp_code)

In [66]:
BATCH_SIZE = 1
newsloader = DataLoader(news_set, batch_size=BATCH_SIZE, shuffle = False)
BATCH_SIZE = 16
dartloader = DataLoader(dart_set, batch_size=BATCH_SIZE, shuffle = False)

# val_dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE)


In [67]:
class BertEncoder(BertPreTrainedModel):

    def __init__(self, config):
        super(BertEncoder, self).__init__(config)
        self.bert = BertModel(config)
        self.init_weights()
    
    def forward(self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [68]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [69]:
news_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)
dart_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.tr

In [70]:
import heapq 
first_b = next(iter(newsloader))
top5 = []
dart_emb_list = []
with torch.no_grad():
    news_emb = news_encoder( first_b[0].to(device), first_b[1].to(device), first_b[2].to(device) ) # (1, emb)

    for i, b in enumerate(dartloader):
        
        dart_emb = news_encoder( b[0].to(device), b[1].to(device), b[2].to(device) ) # (B, emb)

        sim_score_mat = torch.matmul(dart_emb, news_emb.T ) # (B, 1)
        sim_score_vec = sim_score_mat.view(-1)
        max_idx = torch.argmax(sim_score_vec)
        
        top5.extend(sim_score_vec.tolist())
        if i % 100 == 0:
            print(i)
            # print(sim_score_vec.shape)
        del dart_emb
        torch.cuda.empty_cache()
    del news_emb

top5[:5]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200


[300.42413330078125,
 297.0402526855469,
 297.3707580566406,
 298.8297424316406,
 297.8077697753906]

In [71]:
len(top5), top5[:10]

(20684,
 [300.42413330078125,
  297.0402526855469,
  297.3707580566406,
  298.8297424316406,
  297.8077697753906,
  297.5516662597656,
  294.0634765625,
  294.519775390625,
  296.3714904785156,
  298.7331237792969])

In [72]:
len(dartloader) * BATCH_SIZE

20688

In [73]:
top5_tuple = []
for i, t in enumerate(top5):
    top5_tuple.append( (i, t) )


top5_tuple = list(sorted(top5_tuple, key = lambda x:x[1]))
top_idx = top5_tuple[0][0]



In [76]:
train_df.iloc[0]['divided']

'여수광양항만공사(사장 박희석)는 14일부터 20일까지 광양항 배후 투자 유치를 위해 CEO가 직접 참여하는 투자유치 활동을 중국 중남부 지역에서 펼친다. 자유 구역청과 합동으로 진행되는 번 투자유치 활동은 방희선 사장이 직접 중국 현지의 투자의 향 기업을 방문해 광양항 배후 단지의 장점 등을 소개하고 투자협약(MOU)을 체결하는 방식으로 진행된다. 방 사장은 먼저 광양항 서측 배후 푸드존 투자유치를 위해 중국 윈난성 소재 커피 원재료 공급업체인 운남허메이격치(주)를 방문해 광양항 배후 투자협약을 체결할 예정이다. 투자유치 활동은 지난해 9월 경제청과 공사가 합동으로 중국 지역 투자유치 활동을 통해 투자의 향 기업을 선정했던 후속 조치로, 번 방문을 통해 광양항 배후 단지에 실질적인 투자가 이뤄질 수 있도록 '

In [78]:
for t in top5_tuple[:5]:
    print( split_dart_df.iloc[t[0]]['txt'] )

7 3 성장했으며 2017 2022년까지 연평균 7 6 성장하면서 6902억 달러에 달할 것으로 전망되었습니다 세계 IT 서비스 시장 규모는 2018년 6942억 달러로 전년 대비 3 2 성장하였으며 2017 2022년까지 연평균 3 8 성장하면서 7835억 달러에 달할 것으로 전망되었습니다 세계 게임 SW 시장 규모는 2018년 1379억 달러로 전년 대비 13 3 성장하였으며 2017 2021년까지 연평균 10 3 성장하면서 1801억 달러에 달할 것으로 전망되었습니다 나 국내 시장 전망 글로벌 시장과 마찬가지로 국내 소프트웨어 시장도 견조한 성장세를 유지하고 있으며 지속적으로 성장세를 이어갈 것으로 전망됩니다 2018 국내 SW수출 연평균 성장률은 게임SW 부문이 24 7 로 전체 연평균 성장률 11 2 를 크게 웃돌았으며 IT서비스는 3 8 패키지 SW는 7 1 성장하며 전체 성장률보다 낮은 성장을 보였습니다 2018년 국내 SW시장은 전년 대비 4 2 성장하여 24조 원 시
당사의 사업은 마트글라스 투명전광유리 판매 사업과 비메모리반도체의 일종인 CMOS이미지센서의 개발과 판매 사업입니다
가 사업부문별 영업내용 나 사업부문별 요재무정보
섬유제품 제조판매 및 임대사업
골판지 상자 매출액비율 100 의 매출구조를 갖춘 골판지용지 일관생산 전문업체로서 별도의 사업부문은 구분하지 않습니다 조직도


In [223]:
from torch import optim
news_optimizer = optim.SGD(news_encoder.parameters(), lr=0.01, momentum=0.9)
dart_optimizer = optim.SGD(dart_encoder.parameters(), lr=0.01, momentum=0.9)

from torch.nn import CrossEntropyLoss

In [ ]:
from collections import Counter

epochs = 3
for e in range( epochs ):
    e_loss = 0
    for batch in tq.tqdm( dataloader ):
        news_data = {
            'input_ids' : batch[0].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[1].to(device).to(torch.int64),
            'attention_mask' : batch[2].to(device).to(torch.int64),
        }
        
        dart_data = {
            'input_ids' : batch[3].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[4].to(device).to(torch.int64),
            'attention_mask' : batch[5].to(device).to(torch.int64),
        }

        corp_table = batch[6]


        corp_code = corp_table.tolist()
        dic = {}


        batch_size = batch[0].shape[0]
        del batch
        
        news_encoder.train()
        news_encoder.zero_grad()
        news_emb = news_encoder(**news_data) # (B, hidden_dim)
        dart_encoder.train()
        dart_encoder.zero_grad()
        dart_emb = dart_encoder(**dart_data) # (B, hidden_dim)
        sim_score = torch.matmul( news_emb, dart_emb.T ) # (B , B)
        
        target = torch.arange(start=0, end = batch_size).to(device).to(torch.int64)
    
        CE_loss = CrossEntropyLoss()
        loss = CE_loss(sim_score, target)
        e_loss += loss.item()

        loss.backward()
        dart_optimizer.step()
        news_optimizer.step()

        del dart_emb
        del news_emb
        torch.cuda.empty_cache()
    print(e_loss / len(dataloader))